In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bestmodel/kaggle/working/distilgpt2/config.json
/kaggle/input/bestmodel/kaggle/working/distilgpt2/training_args.bin
/kaggle/input/bestmodel/kaggle/working/distilgpt2/model.safetensors
/kaggle/input/bestmodel/kaggle/working/distilgpt2/generation_config.json
/kaggle/input/dataset/kaggle/working/test/state.json
/kaggle/input/dataset/kaggle/working/test/dataset_info.json
/kaggle/input/dataset/kaggle/working/test/dataset.arrow


In [2]:
import re

def preprocess_text(text):
    # Menghapus @mentions
    text = re.sub(r'@\w+', '', text)

    # Menghapus URL
    text = re.sub(r'http\S+|www.\S+', '', text)

    # Menghapus emoticon
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)

    # Menghapus karakter non-ASCII
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Menghapus karakter tertentu
    text = re.sub(r'[\^*\\~-].*', '', text)

    # Menghapus new line
    text = re.sub(r'\n', ' ', text)

    return text

In [3]:
import spacy
from typing import List, Dict, Any, Set, Tuple
# Install packages as needed
!python -m spacy download en_core_web_sm
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')
tokenizer_pattern: str = r"\b\w[\w']*\b"

stemmer = PorterStemmer()

def tokenize_text_en(text: str, tokenizer_pattern: str) -> List[str]:
  tokens: List[str] = re.findall(tokenizer_pattern, text)
  return tokens
# Lemmatize
nlp = spacy.load("en_core_web_sm")

def lemmatize_tokens_en(tokens: List[str], nlp) -> List[str]:
  example_lemmatized_en = []

  for doc in nlp.pipe(tokens):
    tok = [token.lemma_ for token in doc]
    example_lemmatized_en.extend(tok)

  return example_lemmatized_en

def stem_tokens_en(tokens: List[str], stemmer: PorterStemmer) -> List[str]:
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  return stemmed_tokens

nltk_stop_words_list: List[str] = stopwords.words('english')
nltk_stop_words_set: Set[str] = set(nltk_stop_words_list)

def remove_stop_words_en(tokens: List[str], stop_words: Dict[str, Any]) -> List[str]:
  tokens_without_stop_words: List[str] = [
      token
      for token in tokens
      if token not in stop_words
  ]
  return tokens_without_stop_words

def join_words_en(tokens: List[str]) -> str:
  words: str = ' '.join(tokens)
  return words


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
def preprocess_text_en(text: str,
                                   stemmer: PorterStemmer,
                                   tokenizer_pattern: str,
                                   nlp) -> str:
  tokens: List[str] = tokenize_text_en(
    text = text,
    tokenizer_pattern = tokenizer_pattern,
  )
  tokens: List[str] = lemmatize_tokens_en(
    tokens = tokens,
    nlp = nlp,
  )
  tokens: List[str] = stem_tokens_en(
    tokens = tokens,
    stemmer = stemmer,
  )
  tokens: List[str] = remove_stop_words_en(
    tokens = tokens,
    stop_words = nltk_stop_words_set,
  )
  words: str = join_words_en(
      tokens = tokens,
  )
  return words

In [5]:
inp = input("Chat: ")
prep1 = preprocess_text(inp)
print(f'prep1 {prep1}')
prep2 = preprocess_text_en(prep1, stemmer, tokenizer_pattern, nlp)
print(f'prep2 {prep2}')

Chat:  halo my name is blabla \n


prep1 halo my name is blabla 
prep2 halo name blabla


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
import torch
from torch import nn
from transformers import set_seed

device: torch.device = torch.device("cuda") \
  if torch.cuda.is_available() else torch.device("cpu")
instruction_format: str = (
    "Below is an instruction that describes a task. "
    "Write a response that appropriately completes the request.\n"
    "\n"
    "### Question:\n"
    "{question}"
    "\n\n"
    "### Answer:\n"
    "{answer}"
)
    
tokenizer: PreTrainedTokenizerBase = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token
model: nn.Module = AutoModelForCausalLM.from_pretrained('/kaggle/input/bestmodel/kaggle/working/distilgpt2')
model.to(device)
print(f"model: {model}")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model: GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [6]:
def preprocess_answer(text):
  split_output = text.split('### Answer:')
  answer_split = split_output[1].split(' ')
  answer = ''
  index=0
  for i in answer_split:
      if i!='':
          index+=1
      if index > 2 and i=='':
          break
      else:
          answer = answer+i+' '
  answer = answer.split()
  answer = ' '.join(answer)
  return answer

In [7]:
set_seed(87)
def generate_text_sampling_top_k_50(
    input_prompt: str,
    min_length: str = 3,
    max_length: int = 500,
    top_k: int = 50,
  ) -> str:
  input_prompt: str = instruction_format.format(
      question=input_prompt,
      answer='',
  )
  encoded_input: BatchEncoding = tokenizer(input_prompt, return_tensors='pt').to(device)
  sampling_output_tensor: Tensor = model.generate(
      **encoded_input,
      min_length=min_length,
      max_length=max_length,
      pad_token_id=50256,

      do_sample=True,
      top_k=top_k,
  )
  sampling_output_text: str = tokenizer.batch_decode(sampling_output_tensor, skip_special_tokens=True)[0]
  answer = preprocess_answer(sampling_output_text)
  return answer



2024-02-29 02:47:20.260784: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 02:47:20.260897: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 02:47:20.387993: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
set_seed(87)
def generate_text_sampling_top_k_10(
    input_prompt: str,
    min_length: str = 3,
    max_length: int = 256,
    top_k: int = 10,
  ) -> str:
  input_prompt: str = instruction_format.format(
      question=input_prompt,
      answer='',
  )
  encoded_input: BatchEncoding = tokenizer(input_prompt, return_tensors='pt').to(device)
  sampling_output_tensor: Tensor = model.generate(
      **encoded_input,
      min_length=min_length,
      max_length=max_length,
      pad_token_id=50256,
      do_sample=True,
      top_k=top_k,
  )
  sampling_output_text: str = tokenizer.batch_decode(sampling_output_tensor, skip_special_tokens=True)[0]
  answer = preprocess_answer(sampling_output_text)
  return answer



In [9]:
set_seed(87)
def generate_text_sampling_top_p_nucleus_22(
    input_prompt: str,
    min_length: int = 3,
    max_length: int =256,
    top_p: float = 0.22,
  ) -> str:
  input_prompt: str = instruction_format.format(
      question=input_prompt,
      answer='',
  )
  encoded_input: BatchEncoding = tokenizer(input_prompt, return_tensors='pt').to(device)
  sampling_output_tensor: Tensor = model.generate(
      **encoded_input,
      min_length=min_length,
      max_length=max_length,
      pad_token_id=50256,
      do_sample=True,
      top_p=top_p,
      top_k=0,
  )
  sampling_output_text: str = tokenizer.batch_decode(sampling_output_tensor, skip_special_tokens=True)[0]
  answer = preprocess_answer(sampling_output_text)
  return answer


In [10]:
set_seed(87)
def generate_text_sampling_top_p_nucleus_92(
    input_prompt: str,
    min_length: int = 3,
    max_length: int =256,
    top_p: float = 0.92,
  ) -> str:
  input_prompt: str = instruction_format.format(
      question=input_prompt,
      answer='',
  )
  encoded_input: BatchEncoding = tokenizer(input_prompt, return_tensors='pt').to(device)
  sampling_output_tensor: Tensor = model.generate(
      **encoded_input,
      min_length=min_length,
      max_length=max_length,
      do_sample=True,
      pad_token_id=50256,
      top_p=top_p,
      top_k=0,
  )
  sampling_output_text: str = tokenizer.batch_decode(sampling_output_tensor, skip_special_tokens=True)[0]
  answer = preprocess_answer(sampling_output_text)
  return answer


In [11]:
def preprocess_question(text):
    prep1 = preprocess_text(text)
    return preprocess_text_en(prep1, stemmer, tokenizer_pattern, nlp)

In [13]:
lol = 'i already dm my email'
quest = preprocess_question(lol)
print(f'prep2 {quest}')
print(f'answer: {generate_text_sampling_top_p_nucleus_22(quest)}')

prep2 I alreadi dm email
answer: We've responded to your DM. Thank you.
